In [4]:
import json
import nltk
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer

In [5]:
#load the corpus as a string

source_text = ""

with open("tweets.csv", "r") as f:
    for line in f:
        source_text += line
        source_text += " "

In [6]:
#split string into sentences

tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
# punctuations = list(string.punctuation)

raw_sentences = tokenizer.tokenize(source_text)


In [7]:
raw_sentences[212]

'#SeduceMeIn4Words\n want to come inside?'

In [8]:
#clean each sentence, saving semantically meaningful words into an array

substring = "http"
substring2 = ".com"

def sentence_to_wordlist(raw):
    stopWords = set(stopwords.words('english'))
    words = raw.lower().split()
    sentenceArray = []

    for word in words:
        if word in stopWords or substring in word or substring2 in word:
            continue
        else:
            sentenceArray.append(word)
            
    # Create a list of words
    return sentenceArray

In [9]:
cleaned_sentences = []
for raw in raw_sentences:
    cleaned_sentences.append(sentence_to_wordlist(raw))

In [10]:
print(cleaned_sentences[212])

['#seducemein4words', 'want', 'come', 'inside?']


In [11]:
#build the w2v model

model = w2v.Word2Vec(sg=1,seed=1,workers= 4,size=100,min_count=3,window=15,sample=1e-3)
model.build_vocab(cleaned_sentences)
model.train(cleaned_sentences, total_examples=model.corpus_count,epochs=model.iter)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(6391, 24760)

In [12]:
print(model)

Word2Vec(vocab=190, size=100, alpha=0.025)


In [13]:
#save the model as a lookup dictionary json

vectorsDict = {}
for sentence in cleaned_sentences:
    for word in sentence:
        keys = list(vectorsDict.keys())
        if word not in keys:
            try:
                vectorsDict[word] = model.wv[word].tolist()
            except:
                continue

In [14]:
with open("SeduceMeIn4Wordstweets.json", "w+") as f:
    json.dump(vectorsDict, f)